## Numeric Data

The API serves normalized and as-reported financial information in JSON.

Calcbench uses cookie based authentication, you will need to use a client that handles cookies, an example using Python's requests package is below.

In [22]:
import requests
import json
cb_email = "andrew@calcbench.com" # put your calcbench username here
cb_password = "password" # put your calcbench password here

In [8]:
CALCBENCH_SESSION = requests.Session()
CALCBENCH_API_URL_BASE = "https://www.calcbench.com/api/{0}"
r = CALCBENCH_SESSION.post('https://www.calcbench.com/account/LogOnAjax', 
                  {'email' : cb_email, 
                   'password' : cb_password, 
                   'rememberMe' : 'true'}, 
                  verify=True)
assert r.text == 'true', 'login failed'

#### Get tickers by industry.

In [3]:
pharma_SIC_code = 2834
url = CALCBENCH_API_URL_BASE.format("companies?siccodes={0}".format(pharma_SIC_code))
pharma_companies = CALCBENCH_SESSION.get(url).json()
pharma_tickers = [co['ticker'] for co in pharma_companies]

### Normalized Data

In [27]:
metrics = ['revenue', 'netincome', 'assets', 'stockholdersequity']

payload = {'periodParameters' : {'year' : 2010, 'period' : 0, 'endYear' : 2014, 'endPeriod' : 0},
           'companiesParameters' : {'companyIdentifiers' : ['ibm', 'goog']},
           'pageParameters' : {'metrics' : metrics}
           }
url = CALCBENCH_API_URL_BASE.format("/mappedData")
data = CALCBENCH_SESSION.post(url,
                              data=json.dumps(payload), 
                              headers={'content-type' : 'application/json'}
                              ).json()

In [28]:
data

{'Message': 'An error has occurred.'}

### As Reported Data

In [6]:
ticker = "ibm"
statement_type = "income" # one of (income, balance, cash)
as_reported_path = 'asreported/?companyIdentifier={0}&statementType={1}&periodType=annual'.format(ticker, statement_type)
response = CALCBENCH_SESSION.get(CALCBENCH_API_URL_BASE.format(as_reported_path), verify=True)
as_reported_data = json.loads(response.text)

In [7]:
as_reported_data

{'columns': [{'PRELIMINARYDATAID': 0,
   'calculated': False,
   'compareType': 'change',
   'date_range': '1/1/2017 to 12/31/2017',
   'fiscal_period': 'Y 2017',
   'fiscal_period_type': 4,
   'instant': False,
   'is_guidance_column': True,
   'period_end': '2017-12-31',
   'period_start': '2017-01-01',
   'preliminary': False,
   'sec_links': []},
  {'PRELIMINARYDATAID': 0,
   'calculated': False,
   'compareType': 'change',
   'date_range': '1/1/2016 to 12/31/2016',
   'fiscal_period': 'Y 2016',
   'fiscal_period_type': 4,
   'instant': False,
   'is_guidance_column': False,
   'period_end': '2016-12-31',
   'period_start': '2016-01-01',
   'preliminary': False,
   'sec_links': [{'document_type': '10-K',
     'filing_date': '2/28/2017',
     'link': 'http://www.sec.gov/Archives/edgar/data/51143/000104746917001061/0001047469-17-001061-index.htm'},
    {'document_type': '8-K',
     'filing_date': '1/19/2017',
     'link': 'http://www.sec.gov/Archives/edgar/data/51143/0001104659170030